In [99]:
import pandas as pd
from pandasql import sqldf
import os
import dtale
import plotly.express as px

In [100]:
df = pd.read_csv('vgsales.csv')

In [101]:
# Show first 5 rows of data

df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


# Data Cleaning and Preparation

### Data Dictionary

* **Rank:** Ranking of overall sales

* **Name:** The games name

* **Platform:** Platform of the games release (i.e. PC,PS4, etc.)

* **Year:** Year of the game's release

* **Genre:** Genre of the game

* **Publisher:** Publisher of the game

* **NA_Sales:** Sales in North America (in millions)

* **EU_Sales:** Sales in Europe (in millions)

* **JP_Sales:** Sales in Japan (in millions)

* **Other_Sales:** Sales in the rest of the world (in millions)

* **Global_Sales:** Total worldwide sales

### Data Assumptions

* There were over 2,200 duplicate rows in the data. After using DTale to filter, I discovered that this was because some games were released on multiple platforms, and therefore left the duplicate rows in.
  
* Under each Sales column, any value shown as 0.00 is under 100,000 dollars sales. Any value shown as 0.01 is over 100,000 dollars in sales.

In [102]:
# Count NaN values in each column
nan_counts = df.isnull().sum()

# Print the count of NaN values for each column
print(nan_counts)

Rank             0
Name             0
Platform         0
Year            13
Genre            0
Publisher        0
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
Global_Sales     0
dtype: int64


In [103]:
#### Using Pandas to add Year values from game titles where the year is included (ex: "Madden NFL 2004")

# Define a function to extract the year from the game title
def extract_year_from_title(name):
    import re
    # Regular expression to find four consecutive digits that likely represent a year
    match = re.search(r'(19\d{2}|20\d{2})', name)
    if match:
        return int(match.group(0))
    return None

# Apply the function to create a new column 'Extracted_Year'
df['Extracted_Year'] = df['Name'].apply(extract_year_from_title)

# Replace NaN values in the 'Year' column with the values from 'Extracted_Year'
df['Year'] = df['Year'].fillna(df['Extracted_Year'])

# Remove the 'Extracted_Year' column, no longer needed
df.drop('Extracted_Year', axis=1, inplace=True)

In [104]:
# Count NaN values in each column
nan_counts = df.isnull().sum()

# Print the count of NaN values for each column
print(nan_counts)

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64


In [105]:
# Write a SQL query to filter less reliable data out of the DataFrame

sqlyear = 'SELECT * FROM df WHERE Year >=1980 AND Year <=2012'

df2 = sqldf(sqlyear)

In [106]:
# Save the modified DataFrame back to a CSV
df2.to_csv('newyear_data.csv', index=False)

In [107]:
df2

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
14408,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
14409,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
14410,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
14411,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [108]:
#SQL query to find the total game revenue from 1980 to 2012

sqltotal = "SELECT sum(Global_Sales) as Total_Revenue FROM df2"

total = sqldf(sqltotal)

total

,Total_Revenue
0,7868.62


# Section 1: Analysis of Historical Physical Sales Revenue (1980-2012)

#### Query 1

In [109]:
# SQL query to aggregate sales data by Number of Games Published vs Sales Per Year
sqlpivot = "SELECT Year, count(Name) as Games_Published, sum(NA_Sales) as NA_Sales, sum(EU_Sales) as EU_Sales, sum(JP_Sales) as JP_Sales, sum(Other_Sales) as Other_Sales, sum(Global_Sales) as Global_Sales FROM df2 GROUP BY 1"

# Run the SQL query
pivot_table_df2 = sqldf(sqlpivot)

In [110]:
# Specify the path for saving a CSV file of the filtered DataFrame
pivot_table_df2.to_csv('pivot_table_df2.csv', index=False)

#### Query 2

In [111]:
#SQL Query to find Total Sales Per Genre 

sqltotalsalesgenre = "Select sum(Global_Sales) as Global_Sales, Genre FROM df2 GROUP BY Global_Sales"

totalsalesgenre = sqldf(sqltotalsalesgenre)

# Specify the path for saving a CSV file of the filtered DataFrame
totalsalesgenre.to_csv('totalsalesgenre.csv', index=False)

In [112]:
totalsalesgenre

,Global_Sales,Genre
0,4.59,Action
1,16.98,Simulation
2,19.44,Strategy
3,21.76,Puzzle
4,26.25,Puzzle
...,...,...
587,31.37,Role-Playing
588,33.00,Sports
589,35.82,Racing
590,40.24,Platform


#### Query 3

In [113]:
#SQL query to find The Top 3 Genres Per Year Over Time


# "WITH RankedGenres AS
#  (SELECT Year, Genre, sum(Global_Sales) AS Global_Sales,
#    RANK() OVER (PARTITION BY Year ORDER BY Global_Sales DESC) AS Rank 
# FROM df2 GROUP BY Year, Genre),

# AggregatedSales AS
#  (SELECT Year, CASE WHEN Rank <= 3 THEN Genre ELSE 'Other' 
#    END AS Display_Genre, sum(Global_Sales) AS Global_Sales 
# FROM RankedGenres GROUP BY Year, 
# CASE WHEN Rank <= 3 THEN Genre ELSE 'Other' END) 

# SELECT Year, Display_Genre, sum(Global_Sales) AS Global_Sales 
# FROM AggregatedSales GROUP BY Year, Display_Genre 
# ORDER BY Year, sum(Global_Sales) DESC;"



sqlgenre = "WITH RankedGenres AS (SELECT Year, Genre, sum(Global_Sales) AS Global_Sales, RANK() OVER (PARTITION BY Year ORDER BY Global_Sales DESC) AS Rank FROM df2 GROUP BY Year, Genre), AggregatedSales AS (SELECT Year, CASE WHEN Rank <= 3 THEN Genre ELSE 'Other' END AS Display_Genre, sum(Global_Sales) AS Global_Sales FROM RankedGenres GROUP BY Year, CASE WHEN Rank <= 3 THEN Genre ELSE 'Other' END) SELECT Year, Display_Genre, sum(Global_Sales) AS Global_Sales FROM AggregatedSales GROUP BY Year, Display_Genre ORDER BY Year, sum(Global_Sales) DESC;"

dfgenre = sqldf(sqlgenre)

# Specify the path for saving a CSV file of the filtered DataFrame
dfgenre.to_csv('dfgenre.csv', index=False)

In [114]:
dtale.show(dfgenre)

#### Query 4

In [115]:
# Similar SQL queries as the previous aggregation, except instead by Publisher

# SQL query to find Total Sales Per Publisher 

sqltotalsalespub = "SELECT Publisher, sum(Global_Sales) AS Global_Sales FROM df2 GROUP BY Publisher ORDER BY Global_Sales DESC LIMIT 10;"

totalsalespub = sqldf(sqltotalsalespub)

# Specify the path for saving a CSV file of the filtered DataFrame
totalsalespub.to_csv('totalsalespub.csv', index=False)

#### Query 5

In [116]:
# SQL query to find The Top 3 Publishers Per Year Over Time, first removing all but the top 10 and labeling the rest as "Other"

# "WITH TotalSales AS
#  (SELECT Publisher, sum(Global_Sales) AS Total_Global_Sales
#   FROM df2 GROUP BY Publisher ORDER BY Total_Global_Sales DESC LIMIT 10),

# YearlySales AS
#  (SELECT Year, Publisher, sum(GlobalSales) AS Yearly_Global_Sales
#  FROM df2 GROUP BY Year, Publisher),

# RankedYearlySales AS 
#  (SELECT Year, Publisher, Yearly_Global_Sales, 
#  RANK() OVER (PARTITION BY Year ORDER BY Yearly_Global_Sales DESC) AS Rank 
#  FROM YearlySales WHERE Publisher IN (SELECT Publisher FROM TotalSales)), 

# FinalSales AS 
#  (SELECT Year, CASE WHEN Rank <= 3 THEN Publisher ELSE 'Other' END AS Display_Publisher, 
#  sum(Yearly_Global_Sales) AS Global_Sales FROM RankedYearlySales GROUP BY Year, 
#  CASE WHEN Rank <= 3 THEN Publisher ELSE 'Other' END) 

# SELECT Year, Display_Publisher, sum(Global_Sales) AS Total_Sales 
# FROM FinalSales GROUP BY Year, Display_Publisher ORDER BY Year, Total_Sales DESC;"

sqlpub = "WITH TotalSales AS (SELECT Publisher, sum(Global_Sales) AS Total_Global_Sales FROM df2 GROUP BY Publisher ORDER BY Total_Global_Sales DESC LIMIT 10), YearlySales AS (SELECT Year, Publisher, sum(Global_Sales) AS Yearly_Global_Sales FROM df2 GROUP BY Year, Publisher), RankedYearlySales AS (SELECT Year, Publisher, Yearly_Global_Sales, RANK() OVER (PARTITION BY Year ORDER BY Yearly_Global_Sales DESC) AS Rank FROM YearlySales WHERE Publisher IN (SELECT Publisher FROM TotalSales)), FinalSales AS (SELECT Year, CASE WHEN Rank <= 3 THEN Publisher ELSE 'Other' END AS Display_Publisher, sum(Yearly_Global_Sales) AS Global_Sales FROM RankedYearlySales GROUP BY Year, CASE WHEN Rank <= 3 THEN Publisher ELSE 'Other' END) SELECT Year, Display_Publisher, sum(Global_Sales) AS Total_Sales FROM FinalSales GROUP BY Year, Display_Publisher ORDER BY Year, Total_Sales DESC;"

dfpublisher = sqldf(sqlpub)

# Specify the path for saving a CSV file of the filtered DataFrame
dfpublisher.to_csv('dfpublisher.csv', index=False)

# Section 2: Analysis of Peak Market Revenue (2008)

#### Query 6

In [117]:
# SQL queries to analyze the peak in videogame sales in 2008

sql2008 = "SELECT * FROM df2 WHERE Year = 2008 GROUP BY Year, Name"

peaksales = sqldf(sql2008)

# Specify the path for saving a CSV file of the filtered DataFrame
peaksales.to_csv('peaksales2008.csv', index=False)


#### Query 7

In [118]:
# Query to analyze peak sales by region

# "SELECT Year, sum(Global_Sales) as Global_Sales, sum(NA_Sales) as NA_Sales, sum(EU_Sales) 
# as EU_Sales, sum(JP_Sales) as JP_Sales, 
# sum(Other_Sales) as Other_Sales 
# FROM peaksales GROUP BY Year"

sqlregion = "SELECT Year, sum(Global_Sales) as Global_Sales, sum(NA_Sales) as NA_Sales, sum(EU_Sales) as EU_Sales, sum(JP_Sales) as JP_Sales, sum(Other_Sales) as Other_Sales FROM peaksales GROUP BY Year"

sqlregdataset = sqldf(sqlregion)

# Specify the path for saving a CSV file of the filtered DataFrame
sqlregdataset.to_csv('sqlregdataset.csv', index=False)


# Section 3: Analysis of Gaming Platform Revenue During Online Marketplace Introduction (2010)

In [119]:
# SQL query to narrow down entries from only 2010

sql2010 = "SELECT * FROM df2 WHERE Year = 2010 GROUP BY Year, Name"

intromarketplace = sqldf(sql2010)

# Specify the path for saving a CSV file of the filtered DataFrame
intromarketplace.to_csv('intromarketplace.csv', index=False)

#### Query 8

In [120]:
# SQL query to find the top 3 consoles, PC, and label all else as "Other"

# WITH RankedPlatforms AS 
# (SELECT Year, Platform, sum(Global_Sales) AS Global_Sales, 
# RANK() OVER (PARTITION BY Year ORDER BY sum(Global_Sales) DESC) AS Rank 
# FROM intromarketplace 
#GROUP BY Year, Platform), 
# AggregatedSales AS 
# (SELECT Year, CASE WHEN Rank <= 3 OR Platform = 'PC' THEN Platform ELSE 'Other' 
# END AS Display_Platform, sum(Global_Sales) AS Global_Sales 
# FROM RankedPlatforms 
# GROUP BY Year, CASE WHEN Rank <= 3 OR Platform = 'PC' THEN Platform ELSE 'Other' END) 
# SELECT Year, Display_Platform, sum(Global_Sales) AS Global_Sales 
# FROM AggregatedSales 
# GROUP BY Year, Display_Platform 
# ORDER BY Year, sum(Global_Sales) DESC;

sqlconsoles2010 = "WITH RankedPlatforms AS (SELECT Year, Platform, sum(Global_Sales) AS Global_Sales, RANK() OVER (PARTITION BY Year ORDER BY sum(Global_Sales) DESC) AS Rank FROM intromarketplace GROUP BY Year, Platform), AggregatedSales AS (SELECT Year, CASE WHEN Rank <= 3 OR Platform = 'PC' THEN Platform ELSE 'Other' END AS Display_Platform, sum(Global_Sales) AS Global_Sales FROM RankedPlatforms GROUP BY Year, CASE WHEN Rank <= 3 OR Platform = 'PC' THEN Platform ELSE 'Other' END) SELECT Year, Display_Platform, sum(Global_Sales) AS Global_Sales FROM AggregatedSales GROUP BY Year, Display_Platform UNION ALL SELECT NULL AS Year, 'Total' AS Display_Platform, sum(Global_Sales) AS Global_Sales FROM AggregatedSales  ORDER BY Year, sum(Global_Sales) ASC;"

consolestop = sqldf(sqlconsoles2010)

# Specify the path for saving a CSV file of the filtered DataFrame
consolestop.to_csv('consolestop.csv', index=False)

consolestop

,Year,Display_Platform,Global_Sales
0,NaN,Total,447.19
1,2010.0,PC,17.55
2,2010.0,PS3,93.19
3,2010.0,Other,100.84
4,2010.0,Wii,113.75
5,2010.0,X360,121.86


#### Query 9

In [121]:
# SQL query to find the top 7 genres and label all else as "Other"

# "WITH RankedGenres AS
#  (SELECT Year, Genre, sum(Global_Sales) AS (Global_Sales),
#    RANK() OVER (PARTITION BY Year ORDER BY Global_Sales DESC) AS Rank 
# FROM intromarketplace GROUP BY Year, Genre),

# AggregatedSales AS
#  (SELECT Year, CASE WHEN Rank <= 7 THEN Genre ELSE 'Other' 
#    END AS Display_Genre, sum(Global_Sales) AS Global_Sales 
# FROM RankedGenres GROUP BY Year, 
# CASE WHEN Rank <= 7 THEN Genre ELSE 'Other' END) 

# SELECT Year, Display_Genre, sum(Global_Sales) AS Global_Sales 
# FROM AggregatedSales GROUP BY Year, Display_Genre 
# ORDER BY Year, sum(Global_Sales) DESC;"

sqlgenre2010 = "WITH RankedGenres AS (SELECT Year, Genre, sum(Global_Sales) AS Global_Sales, RANK() OVER (PARTITION BY Year ORDER BY Global_Sales DESC) AS Rank FROM intromarketplace GROUP BY Year, Genre), AggregatedSales AS (SELECT Year, CASE WHEN Rank <= 7 THEN Genre ELSE 'Other' END AS Display_Genre, sum(Global_Sales) AS Global_Sales FROM RankedGenres GROUP BY Year, CASE WHEN Rank <= 7 THEN Genre ELSE 'Other' END) SELECT Year, Display_Genre, sum(Global_Sales) AS Global_Sales FROM AggregatedSales GROUP BY Year, Display_Genre UNION ALL SELECT NULL AS Year, 'Total' AS Display_Genre, sum(Global_Sales) AS Global_Sales FROM AggregatedSales ORDER BY Year, sum(Global_Sales) ASC;"

genrestop = sqldf(sqlgenre2010)

# Specify the path for saving a CSV file of the filtered DataFrame
genrestop.to_csv('genrestop.csv', index=False)

genrestop

,Year,Display_Genre,Global_Sales
0,NaN,Total,447.19
1,2010.0,Puzzle,10.53
2,2010.0,Simulation,17.54
3,2010.0,Racing,25.50
4,2010.0,Shooter,48.10
5,2010.0,Sports,61.13
6,2010.0,Role-Playing,64.03
7,2010.0,Action,69.88
8,2010.0,Other,150.48


#### Query 10

In [122]:
# SQL query to find the top 7 publishers and label all else as "Other"

# "WITH RankedPublishers AS
#  (SELECT Year, Publisher, sum(Global_Sales) AS Global_Sales,
#    RANK() OVER (PARTITION BY Year ORDER BY Global_Sales DESC) AS Rank 
# FROM intromarketplace GROUP BY Year, Publisher),

# AggregatedSales AS
#  (SELECT Year, CASE WHEN Rank <= 7 THEN Publisher ELSE 'Other' 
#    END AS Display_Publisher, sum(Global_Sales) AS Global_Sales 
# FROM RankedPublishers GROUP BY Year, 
# CASE WHEN Rank <= 7 THEN Publisher ELSE 'Other' END) 

# SELECT Year, Display_Publisher, sum(Global_Sales) AS Global_Sales 
# FROM AggregatedSales GROUP BY Year, Display_Publisher
# ORDER BY Year, sum(Global_Sales) DESC"

sqlpub2010 = "WITH RankedPublishers AS (SELECT Year, Publisher, sum(Global_Sales) AS Global_Sales, RANK() OVER (PARTITION BY Year ORDER BY Global_Sales DESC) AS Rank FROM intromarketplace GROUP BY Year, Publisher), AggregatedSales AS (SELECT Year, CASE WHEN Rank <= 7 THEN Publisher ELSE 'Other' END AS Display_Publisher, sum(Global_Sales) AS Global_Sales FROM RankedPublishers GROUP BY Year, CASE WHEN Rank <= 7 THEN Publisher ELSE 'Other' END) SELECT Year, Display_Publisher, sum(Global_Sales) AS Global_Sales FROM AggregatedSales GROUP BY Year, Display_Publisher UNION ALL SELECT NULL AS Year, 'Total' AS Display_Publisher, sum(Global_Sales) AS Global_Sales FROM AggregatedSales ORDER BY Year, sum(Global_Sales) DESC"

pubtop = sqldf(sqlpub2010)

# Specify the path for saving a CSV file of the filtered DataFrame
pubtop.to_csv('pubtop2010.csv', index=False)

pubtop

,Year,Display_Publisher,Global_Sales
0,NaN,Total,447.19
1,2010.0,Other,325.03
2,2010.0,Microsoft Game Studios,49.16
3,2010.0,Electronic Arts,41.05
4,2010.0,Take-Two Interactive,20.28
5,2010.0,MTV Games,4.49
6,2010.0,Bethesda Softworks,4.08
7,2010.0,LucasArts,1.55
8,2010.0,Codemasters,1.55


# Section 4: Videogame Industry Impact on Present-Day (2023)

#### Query 11

In [123]:
# With manual lookup, found a dataset showing jobs created per US State in 2023 and a dataset with the population of each state that year.
jobimpact = pd.read_csv('jobimpact.csv')

statepop = pd.read_csv ('StatePop2023.csv')

jobimpact.head()

,State,Intrastate_Impact,Interstate_Impact
0,Alabama,540,648
1,Alaska,60,69
2,Arizona,"1,970","2,560"
3,Arkansas,312,371
4,California,"126,657","172,150"


In [124]:
statepop.head()

,State,Population
0,Alabama,"5,108,468"
1,Alaska,"733,406"
2,Arizona,"7,431,344"
3,Arkansas,"3,067,732"
4,California,"38,965,193"


In [125]:
# Remove the commas from values and convert into float using Pandas

jobimpact['Intrastate_Impact'] = jobimpact['Intrastate_Impact'].str.replace(',', '').astype(float)
statepop['Population'] = statepop['Population'].str.replace(',', '').astype(float)

#### Query 12

In [126]:
# SQL query to perform a join and find the ratio between Intrastate Impact and State Population

# "SELECT a.State, a.Intrastate_Impact, b.Population, 
# sum(a.Intrastate_Impact) / sum(b.Population) AS ratio 
# FROM jobimpact a JOIN statepop b ON a.State = b.State 
# GROUP BY a.State"

sqlratio = "SELECT a.State, a.Intrastate_Impact, b.Population, a.Intrastate_Impact / (b.Population / 100000.0) AS impact_per_100k FROM jobimpact a JOIN statepop b ON a.State = b.State GROUP BY a.State"

ratio = sqldf(sqlratio)

# Specify the path for saving a CSV file of the filtered DataFrame

ratio.to_csv('2023by100kjoin.csv', index=False)